In [2]:
import json
with open('/Users/marcolaureano2/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
pip install yelpapi

Note: you may need to restart the kernel to use updated packages.


In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# Quick Test Query
results = yelp_api.search_query(location='Baltimore, MD',
                                       term='Crab Cake')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])